In [1]:
import pandas as pd

In [12]:
f_all = pd.read_csv("walmart_data.tsv", sep='\t', header = None, nrows = 500000)
f_all.head(15)

,0,1,2,3,4
0,0,"576904375,40347953,44391515,518600320,10055613...",1,9,pretrain
1,0,"10535678,38253470,939553889,554603726,43806652...",2,9,pretrain
2,0,"518600320,537265296,238622168",3,9,pretrain
3,0,"366904159,47232707,847684551,100556137,1030838...",4,9,pretrain
4,0,"576904375,905427925,737995684,457227614,204779...",5,9,pretrain
5,0,"860556227,46491756,523463942,10318312,10055613...",6,9,pretrain
6,0,"675379614,17126957,274330908",7,9,train
7,0,"10533862,19276123,10449104,16419724,257009703,...",8,9,validation
8,0,"576904375,11027608,574967331,520073152,1561063...",9,9,evaluation
9,1,"10311586,47229657,10292659,929259456,556001479...",1,6,pretrain


In [13]:
data = f_all.values.tolist()
pre_train_data = list(filter(lambda x: x[4] == "pretrain" or x[4] == 'train', data))
train_data = list(filter(lambda x: x[4] == "validation", data))
test_data = list(filter(lambda x: x[4] == "evaluation", data))

In [14]:
print(data[-1])

[61365, '391184518,45726041,20659508,36724056,162974158,560670675,20670285,51259112,51259387,16785072,40815497,897030196,252299264,49138787,24180906,24722278,10450002,666731569,145902183,217072817', 1, 6, 'pretrain']


In [20]:
pre_train_dict = dict()
train_dict = dict()
test_dict = dict()
train_bpr_dict = dict()

for line in test_data:
    user = int(line[0])
    item_id = line[1].split(',')
    for i in range(0, len(item_id)):
        item_id[i] = int(item_id[i])
    test_dict[user] = item_id
    
for line in pre_train_data:
    user = int(line[0])
    item_id = line[1].split(',')
    for i in range(0, len(item_id)):
        item_id[i] = int(item_id[i])
    if user not in pre_train_dict.keys():
        pre_train_dict[user] = [int(line[3])-2]
        train_bpr_dict[user] = []
        pre_train_dict[user] += item_id
        train_bpr_dict[user] += item_id
    else:
        pre_train_dict[user] += item_id
        train_bpr_dict[user] += item_id
    
for line in train_data:
    user = int(line[0])
    item_id = line[1].split(',')
    for i in range(0, len(item_id)):
        item_id[i] = int(item_id[i])
    train_dict[user] = item_id
    train_bpr_dict[user] += item_id

        
item_dict = dict() 
item_new_id = 0
for line in data:
    item_id = line[1].split(',')
    for i in range(0, len(item_id)):
        item_id[i] = int(item_id[i])
    for item in item_id:
        if item not in item_dict.keys():
            item_dict[item] = item_new_id
            item_new_id += 1
        
if data[-1][-1] == 'pretrain' or 'train':
    last_user = max(pre_train_dict.keys())
    del pre_train_dict[last_user]
    del train_bpr_dict[last_user]

new_pre_train_dict = dict()
new_train_dict = dict()
new_test_dict = dict()
new_train_bpr_dict = dict()

for (k,v) in pre_train_dict.items():
    item_id = v[1:]
    new_pre_train_dict[k] = [v[0]]
    new_pre_train_dict[k] += [item_dict[item] for item in item_id]
for (k,v) in train_dict.items():
    new_train_dict[k] = [item_dict[item] for item in v]
for (k,v) in train_bpr_dict.items():
    new_train_bpr_dict[k] = [item_dict[item] for item in v]
for (k,v) in test_dict.items():
    new_test_dict[k] = [item_dict[item] for item in v]

In [21]:
f_train = open("./Data/walmart/train.txt", "w")
for key in new_train_dict.keys():
    line = str(key)
    for product in new_train_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_train.write(line)
f_train.close()

f_train_bpr = open("./Data/walmart/train_bpr.txt", "w")
for key in new_train_bpr_dict.keys():
    line = str(key)
    for product in new_train_bpr_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_train_bpr.write(line)
f_train_bpr.close()

f_test = open("./Data/walmart/test.txt", "w")
for key in new_test_dict.keys():
    line = str(key)
    for product in new_test_dict[key]:
        line += ' ' + str(product)
    line += '\n'
    f_test.write(line)
f_test.close()

In [17]:
f_train_mf = open("./Data/walmart/train_mf.txt","w")

for (u_id,ps) in new_pre_train_dict.items():
    n_order = ps[0]
    items = ps[1:]
    u_fre = dict()
    for p in items:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in u_fre.keys():
        fre = "{:.3f}".format(u_fre[p]/float(n_order))
        line = str(u_id)+ ' ' + str(p) + ' ' + str(fre) + '\n'
        f_train_mf.write(line)
f_train_mf.close()

In [18]:
f_fre_test = open("./Data/walmart/fre_test.txt","w")
for (u_id,ps) in new_pre_train_dict.items():
    n_order = ps[0]
    items = ps[1:]
    u_fre = dict()
    for p in items:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in new_train_dict[u_id]:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in u_fre.keys():
        fre = "{:.3f}".format(u_fre[p]/float(n_order+1))
        line = str(u_id)+ ' ' + str(p) + ' ' + str(fre) + '\n'
        f_fre_test.write(line)
f_fre_test.close()

In [ ]:
recall = 0
for (u_id,ps) in new_pre_train_dict.items():
    n_order = ps[0]
    items = ps[1:]
    u_fre = dict()
    for p in items:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in new_train_dict[u_id]:
        if p not in u_fre.keys():
            u_fre[p] = 1
        else:
            u_fre[p] += 1
    for p in u_fre.keys():
        fre = "{:.3f}".format(u_fre[p]/float(n_order+1))
        line = str(u_id)+ ' ' + str(p) + ' ' + str(fre) + '\n'
        f_fre_test.write(line)